In [1]:
import pandas as pd
import numpy as np

In [2]:
BASEDIR = 'jigsaw-multilingual-toxic-comment-classification'
# BASEDIR = 'Data/Data/'

In [3]:
df = pd.read_csv(BASEDIR + '/jigsaw-unintended-bias-train-processed-seqlen128.csv')

In [4]:
df[:1]

,id,comment_text,toxic,severe_toxicity,obscene,identity_attack,insult,threat,asian,atheist,...,wow,sad,likes,disagree,sexual_explicit,identity_annotator_count,toxicity_annotator_count,input_word_ids,input_mask,all_segment_id
0,59848,"This is so cool. It's like, 'would you want yo...",0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,...,0,0,0,0,0.0,0,4,"(101, 10747, 10124, 10380, 67420, 119, 10377, ...","(1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [5]:
df = df[['comment_text', 'toxic']]

In [6]:
df.loc[df['toxic'] <= 0.5, 'is_toxic'] = 0
df.loc[df['toxic'] > 0.5, 'is_toxic'] = 1 

In [7]:
df.head()

,comment_text,toxic,is_toxic
0,"This is so cool. It's like, 'would you want yo...",0.000000,0.0
1,Thank you!! This would make my life a lot less...,0.000000,0.0
2,This is such an urgent design problem; kudos t...,0.000000,0.0
3,Is this something I'll be able to install on m...,0.000000,0.0
4,haha you guys are a bunch of losers.,0.893617,1.0


In [8]:
df = df.drop('toxic', axis = 1)

In [9]:
df.head()

,comment_text,is_toxic
0,"This is so cool. It's like, 'would you want yo...",0.0
1,Thank you!! This would make my life a lot less...,0.0
2,This is such an urgent design problem; kudos t...,0.0
3,Is this something I'll be able to install on m...,0.0
4,haha you guys are a bunch of losers.,1.0


In [ ]:
!python -m textblob.download_corpora

In [10]:
import numpy as np
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.comment_text, df.is_toxic, test_size=0.30, random_state=42)

In [11]:
X_train

505041     So it must have been women who did the 4 out o...
1689084    There will always be haters in the world but i...
952603     I don't think being an "anti-racist" is one wh...
1246358    maybe putin won't live that much longer then h...
1887591    Ms. Pattis, thank you for saying what should b...
                                 ...                        
259178     Trump's a buffoon, Hillary her staff, and her ...
1414414    She was off? Jeez... then when was she ever on...
131932     Living there for the American military is noth...
671155     Higher trading revenue? Its amazing what doing...
121958     Portland voters made a major mistake when they...
Name: comment_text, Length: 1331535, dtype: object

In [12]:
name = df.is_toxic.name

In [13]:
from textblob import TextBlob
import nltk
def split_into_lemmas(message):
    message = str(message.encode('utf-8')).lower()
    words = TextBlob(message).words
    return [word.lemma for word in words]

In [16]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline

In [22]:
!pip install nltk

In [21]:

text_clf_SGDClassifier = Pipeline([('vect', CountVectorizer(analyzer=split_into_lemmas, ngram_range=(2,4), stop_words='english',lowercase=True)),
                     ('tfidf', TfidfTransformer()),
                     ('clf', SGDClassifier()),
], verbose = True)

In [24]:
text_clf_SGDClassifier.fit(X_train, y_train)

[Pipeline] .............. (step 1 of 3) Processing vect, total=20.1min
[Pipeline] ............. (step 2 of 3) Processing tfidf, total=   4.3s
[Pipeline] ............... (step 3 of 3) Processing clf, total=   3.0s


Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer=<function split_into_lemmas at 0x7fd43b3f07b8>,
                                 binary=False, decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(2, 4), preprocessor=None,
                                 stop_words='english', strip_accents=None,
                                 token_pattern='(?u)\\...
                ('clf',
                 SGDClassifier(alpha=0.0001, average=False, class_weight=None,
                               early_stopping=False, epsilon=0.1, eta0=0.0,
                               fit_intercept=True, l1_ratio=0.15,
                               learning_rate='optimal', loss='hinge',
                               max_iter=1000, n_iter_

In [25]:
predicted_SGDClassifier = text_clf_SGDClassifier.predict(X_test)
print(classification_report(y_test, predicted_SGDClassifier))

              precision    recall  f1-score   support

         0.0       0.95      1.00      0.97    536635
         1.0       0.96      0.11      0.20     34024

    accuracy                           0.95    570659
   macro avg       0.95      0.55      0.58    570659
weighted avg       0.95      0.95      0.93    570659



In [26]:
y.unique()

array([0., 1.])

In [ ]:
text_clf_SVC = Pipeline([('vect', CountVectorizer(analyzer=split_into_lemmas, ngram_range=(2,4), stop_words='english',lowercase=True)),
                     ('tfidf', TfidfTransformer()),
                     ('clf', SVC()),
], verbose = True)
text_clf_SVC.fit(X_train, y_train)

[Pipeline] .............. (step 1 of 3) Processing vect, total=20.0min
[Pipeline] ............. (step 2 of 3) Processing tfidf, total=   5.1s


In [ ]:
predicted_SVC = text_clf_SVCB.predict(X_test)
print (classification_report(y_test, predicted_SVC))

In [17]:
text_clf_MultinomialNB = Pipeline([('vect', CountVectorier(analyzer=split_into_lemmas, ngram_range=(2,4), stop_words='english',lowercase=True)),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultinomialNB()),
], verbose = True)
text_clf_MultinomialNB.fit(X_train, y_train)

[Pipeline] .............. (step 1 of 3) Processing vect, total=20.0min
[Pipeline] ............. (step 2 of 3) Processing tfidf, total=   6.4s
[Pipeline] ............... (step 3 of 3) Processing clf, total=   0.6s


Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer=<function split_into_lemmas at 0x7fd43b3f07b8>,
                                 binary=False, decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(2, 4), preprocessor=None,
                                 stop_words='english', strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('clf',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=True)

In [18]:
from sklearn.metrics import classification_report
predicted_MultinomialNB = text_clf_MultinomialNB.predict(X_test)
print (classification_report(y_test, predicted_MultinomialNB))

              precision    recall  f1-score   support

         0.0       0.94      1.00      0.97    536635
         1.0       0.87      0.01      0.01     34024

    accuracy                           0.94    570659
   macro avg       0.90      0.50      0.49    570659
weighted avg       0.94      0.94      0.91    570659



In [19]:
text_clf_LogisticRegression = Pipeline([('vect', CountVectorizer(analyzer=split_into_lemmas, ngram_range=(2,4), stop_words='english',lowercase=True)),
                     ('tfidf', TfidfTransformer()),
                     ('clf', LogisticRegression()),
])
text_clf_LogisticRegression.fit(X_train, y_train)

/home/fmi-pc-lt-17/Documents/Kaggle/Multilingual Sentiment Analysis/.venv/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer=<function split_into_lemmas at 0x7fd43b3f07b8>,
                                 binary=False, decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(2, 4), preprocessor=None,
                                 stop_words='english', strip_accents=None,
                                 token_pattern='(?u)\\...
                                 tokenizer=None, vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('clf',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
                                    fit_inte

In [20]:
predicted_LogisticRegression = text_clf_LogisticRegression.predict(X_test)

print (classification_report(y_test, predicted_LogisticRegression))

              precision    recall  f1-score   support

         0.0       0.97      0.99      0.98    536635
         1.0       0.79      0.47      0.59     34024

    accuracy                           0.96    570659
   macro avg       0.88      0.73      0.79    570659
weighted avg       0.96      0.96      0.96    570659



In [31]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
text_clf_decisionTree = Pipeline([('vect', CountVectorizer(analyzer=split_into_lemmas, ngram_range=(2,4), stop_words='english',lowercase=True)),
                     ('tfidf', TfidfTransformer()),
                     ('clf', DecisionTreeClassifier()),
], verbose = True)
text_clf_decisionTree.fit(X_train, y_train)

[Pipeline] .............. (step 1 of 3) Processing vect, total=20.3min
[Pipeline] ............. (step 2 of 3) Processing tfidf, total=   7.0s
